
# Test distance law

This notebook tests if Mitsuba3 can reproduce the distance law. For omnidirectional sound sources, the sound power is defined as

$$ P = \int I \, \mathrm{d} S .$$

For omnidirectional sources, the intensity is constant on spheres around the sound sources, which reduces the integral to

$$ P = I \cdot 4 \pi r^2 .$$

The intensity, which is proportional to the sound energy, decreases with the squared distance.

$$ I = \frac{P}{4 \pi r^2} \Rightarrow E \sim \frac{1}{r^2}$$

Let's test this behavior in Mitsuba3:


In [ ]:
import numpy as np
from os import path
import matplotlib.pyplot as plt
import drjit as dr
import mitsuba as mi
from platform import system

if system() == 'Darwin':
    mi.set_variant('llvm_ad_acoustic')
else:
    mi.set_variant('cuda_ad_acoustic')
print(f'Mitsuba variant: {mi.variant()}')

from mitsuba import ScalarTransform4f as T


# set retina backend for plotting with matplotlib magic
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')

mi.set_log_level(mi.LogLevel.Warn)

# Microphone and speaker positions

In [ ]:
mic_pos = (0, 0, 0) # microphone position

N = 30 # number of renderings
distance = np.linspace(1, 10, N)   # distances from source to mic

In [ ]:
config_acoustics = {
    'speed_of_sound': 343.0,
    'speaker_radius': 0.3,

    'integrator': 'prb_acoustic',
    'wav_bins': 1,                  # only one wav bin because we are not interested in frequency-dependent results
    'max_depth': 1,                 # maximum number of bounces, 1 is enough because there is only direct sound
    'max_time':  0.05,
    'time_bins': 1,                 # only one time bin because we want to sum up all the energy anyway
    'spp': 2**18,                   # samples per pixel -> total number of rays because there is only one pixel

}

Dictionary that we will use to build the acoustic scene, all objects, materials and parameters are defined in here.

The acoustic scene uses an acoustic integrator, the acoustic BSDFs and the actual speaker and microphone as emitter and sensor, respectively. Otherwise, the scene definition is the same as the visual scene.

In [ ]:
scene_dict_acoustic = {
    'type': 'scene',

    'integrator': {
        'type': config_acoustics['integrator'],
        'max_depth': config_acoustics['max_depth'],
        'max_time': config_acoustics['max_time'],
        'speed_of_sound': config_acoustics['speed_of_sound'],
        'skip_direct': False,
    },

    'microphone': {
        'type': 'microphone',
        'to_world': T.translate([0, 0, 0]),
        'film': {
            'type': 'tape',
            'wav_bins':  config_acoustics['wav_bins'],
            'time_bins': config_acoustics['time_bins'],
            'rfilter': {'type': 'box'},
            'count': True
        },
        'sampler': {'type': 'stratified', 'sample_count': config_acoustics['spp']},
    },

    'speaker': {
        'type': 'sphere',
        'radius': config_acoustics['speaker_radius'],
        'center': (1, 0, 0),
        'emitter': {'type': 'area', 'radiance': {'type': 'uniform', 'value': 100.}},

    },
}

scene = mi.load_dict(scene_dict_acoustic)

Render the scene. Note that for acoustic path tracing we need to set `spp` bin a lot higher than in the visual rendering to get good results. 

In [ ]:
energy_total = np.zeros_like(distance)

for i, dist in enumerate(distance):
    scene_dict_acoustic['speaker']['center'] = (dist, 0, 0)
    scene = mi.load_dict(scene_dict_acoustic)
    data = mi.render(scene, seed=0, spp=config_acoustics['spp'])
    hist = data[:, :, 0] / config_acoustics["spp"]

    energy_total[i] = np.sum(hist) # the sum is redundant because there is only one time bin



In [ ]:
distance_fine = np.linspace(1, 10, 150)

plt.plot(distance_fine, 1/np.square(distance_fine), label=r'$1/r^2$')
plt.plot(distance, energy_total / energy_total[0], 'o', label='Rendering')
plt.xlabel('Distance in m')
plt.ylabel(r'$E / E_\mathrm{max}$')
plt.legend()
plt.show()